# Loading data and initializing parameters

We set the parameters to load data from our dataset:
- The sampling frequency (100Hz or 500Hz)
- The folder in wich we can retrieve the CSV (labels of the ECG graphs and their datas)
- The type of task (necessary for the code to run but only used for finetuning the model)
- The number of classes (i.e. how many diseases are we trying to predict, 71 is the maximum for the PTBXL dataset)
- The input shape (here we have 10 seconds of record and the standard 12-lead ECG, so input_shape=[sampling_freq * 10, 12])

In [1]:
from utils import utils

sampling_frequency=100
datafolder='../data/ptbxl/'
task='all'

data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)

num_classes = 71         # <=== number of classes in the finetuning dataset
input_shape = [1000,12] # <=== shape of samples, [None, 12] in case of different lengths

# Loading the pretrained model

We load the pretrained model **'xresnet1d101'**, which shows the best results for our task (92.5% overall accuracy)

In [2]:
from models.fastai_model import fastai_model

experiment = 'exp0'
modelname = 'fastai_xresnet1d101'
pretrainedfolder = '../output/'+experiment+'/models/'+modelname+'/'
mpath='../output/'+experiment+'/models/'+modelname+'/' # path where the finetuned model will be stored
n_classes_pretrained = 71 # the number of classes the model has been trained on

model = fastai_model(
    modelname, 
    num_classes, 
    sampling_frequency, 
    mpath, 
    input_shape=input_shape, 
    pretrainedfolder=pretrainedfolder,
    n_classes_pretrained=n_classes_pretrained, 
    pretrained=True,
    epochs_finetuning=2,
)

# Preprocess data with pretrained Standardizer

Since the model was trained with inputs standardized to zero mean and unit variance, our data needs to be standardized with the respective mean and variance. This is also provided in the respective experiment folder `output/expX/data/standard_scaler.pkl`

In [3]:
import pickle

standard_scaler = pickle.load(open('../output/'+experiment+'/data/standard_scaler.pkl', "rb"))

X_val = data[labels.strat_fold == 11]
X_val = utils.apply_standardizer(X_val, standard_scaler)

# Prediction

In [4]:
y_val_pred = model.predict(X_val)

model: fastai_xresnet1d101


aggregating predictions...


# Sending result to the API

We calculate rounded percentage prediction for the 44th class, wich corresponds to the disease **Left Ventrical Hypertrophy**, for one of the predicted ECG.  
Then, we send it to the server via our Django API endpoint.

In [15]:
import requests

testing_index = 5
percent_prediction = round(y_val_pred[testing_index][43] * 100, 1)
print(percent_prediction)

requests.put("http://lauzhack.grelgp.com/set_ecg_value", json = {'ecg_value': str(percent_prediction)})

# Plotting the ECG
import matplotlib.pyplot as plt
plot_vals=[x[0] for x in X_val[testing_index]]

plt.plot(plot_vals)
plt.show()

requests.post("http://lauzhack.grelgp.com/create_img", json = {'plot_vals': plot_vals})

90.9
